- https://nijianmo.github.io/amazon/index.html
- https://jmcauley.ucsd.edu/data/amazon/ 
- https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV

# get data

    download from http://localhost:8080/notebooks/git/product-category/notebooks/secrets_get_amazon_data.ipynb

In [1]:
import pandas as pd
import gzip
import json

In [8]:
pdata = "/data/git/product-category/data/amazon_meta_data"
!ls -hl {pdata}
fnms = !ls {pdata}
fnms = [o for o in fnms if o.endswith('.json.gz')]
domains = [o.strip('meta_').strip('.json.gz') for o in fnms]
dmn2fnm = dict(zip(domains, fnms))
dmn2fnm

total 13G
-rw-rw-r-- 1 ubuntu ubuntu   30M Dec  2 00:07 meta_AMAZON_FASHION.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  9.6M Aug  7  2020 meta_All_Beauty.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  125M Aug  7  2020 meta_Arts_Crafts_and_Sewing.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Aug  7  2020 meta_Automotive.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  153M Aug  7  2020 meta_CDs_and_Vinyl.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  333M Aug  7  2020 meta_Cell_Phones_and_Accessories.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Dec  2 00:18 meta_Clothing_Shoes_and_Jewelry.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   12M Aug  7  2020 meta_Digital_Music.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Electronics.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  37

{'AMAZON_FASHION': 'meta_AMAZON_FASHION.json.gz',
 'All_Beauty': 'meta_All_Beauty.json.gz',
 'Appliance': 'meta_Appliances.json.gz',
 'Arts_Crafts_and_Sewi': 'meta_Arts_Crafts_and_Sewing.json.gz',
 'Automotive': 'meta_Automotive.json.gz',
 'Book': 'meta_Books.json.gz',
 'CDs_and_Vinyl': 'meta_CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessorie': 'meta_Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry': 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music': 'meta_Digital_Music.json.gz',
 'Electronic': 'meta_Electronics.json.gz',
 'Gift_Card': 'meta_Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food': 'meta_Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitche': 'meta_Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific': 'meta_Industrial_and_Scientific.json.gz',
 'Kindle_Store': 'meta_Kindle_Store.json.gz',
 'Luxury_Beauty': 'meta_Luxury_Beauty.json.gz',
 'Magazine_Subscripti': 'meta_Magazine_Subscriptions.json.gz',
 'Movies_and_TV': 'meta_Movies_and_TV.json.gz',
 

In [35]:
KEYS2USE = set(['category', 'description', 'title', 'brand', 'feature', 'asin'])

In [48]:
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7
    
### load the meta data
def get_meta_data(domain, nrows=None):
    keys2use = KEYS2USE
    fnm = dmn2fnm[domain]
    data = []
    with gzip.open(f'{pdata}/{fnm}') as f:
        for i,l in enumerate(f):
            dat = json.loads(l.strip())
            dat = {k:v for k,v in dat.items() if k in keys2use}
            data.append(dat)
            if nrows and i>nrows: break

    # total length of list, this number equals total number of products
#     print("len(data)", len(data))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)
#     print("len(df)", len(df))

    ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)
    df3 = df.fillna('')
    df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
#     print("len(df4)", len(df4))
#     print("len(df5)", len(df5))

    return df5

In [53]:
df = get_meta_data(domains[3], nrows=10000)
df.head(3)

,category,description,title,brand,feature,asin
0,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",[The patch features the sweaty masculine hands...,You Son of a Bitch! 1987 Embroidered Patch,Honchosfx,[You son of a bitch patch - exclusive to Honch...,6665560953
1,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...","[With 5 packs stars folding paper, each pack a...",Origami Stars Papers Package 1H (5 packs),,[],7000000376
2,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",[],Yi De Ge Chinese Calligraphy Sumi Drawing Blac...,MasterChinese,[],7000001089


# preprocess data

In [54]:
dfs = []
for dmn in domains:
    df = get_meta_data(dmn, nrows=1000)
    if set(df.columns)!= KEYS2USE: continue
    df = df[df.category.apply(len)>0]
    if len(df)==0: continue
    print(dmn, df.shape)
    df['domain'] = dmn
    display(df.sample(2))
    dfs.append(df)

Appliance (989, 6)


,category,description,title,brand,feature,asin,domain
854,"[Appliances, Parts &amp; Accessories, Refriger...","[, <br /><strong>Replace Every Six Months</str...","ALPINEWATER 4396841, 4396710, Kenmore 9030, 90...",ALPINEWATER,[The real filter life span depends on water qu...,B000GJYMJK,Appliance
422,"[Appliances, Parts & Accessories, Food Waste D...",[The EZ-FLO garbage disposer splash guard is d...,EZ-FLO 30102 Garbage Disposer Splash Guard,EZ-Flo,"[Splash guard, Fits *insinkerator, Black]",B00068UTY8,Appliance


Arts_Crafts_and_Sewi (971, 6)


,category,description,title,brand,feature,asin,domain
812,"[Arts, Crafts & Sewing, Sewing, Sewing Notions...",[Ideal for carrying bulky items! Heavy duty fu...,VELCRO Brand All-Purpose Strap with Handle | L...,VELCRO Brand,[NOTICE: Amazon is a trusted seller of Velcro ...,B00006IC2S,Arts_Crafts_and_Sewi
831,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...",[Express your creativity! Create announcements...,"Geographics Design Paper, Scroll, 24 lb, 8.5 x...",Geographics,"[Express your creativity!, Create announcement...",B00006IDX0,Arts_Crafts_and_Sewi


Automotive (945, 6)


...

,category,description,title,brand,feature,asin,domain
900,"[Tools & Home Improvement, Power & Hand Tools,...",[Precision Miter Box With Saw - PRECISION MITE...,Jorgensen 64016 Precision Miter Saw,Adjustable Clamp Co,[Miter saw for use in various carpentry projec...,B00002244D,Tools_and_Home_Improvement
606,"[Tools & Home Improvement, Power & Hand Tools,...",[],Bosch 1275DVS 3-Inch x 24-Inch Variable Speed ...,Bosch,"[10.5 Amps, 1,150 SFPM power for professional ...",B0000223H8,Tools_and_Home_Improvement


Toys_and_Game (870, 6)


,category,description,title,brand,feature,asin,domain
590,"[Toys & Games, Learning & Education, Reading &...",[<b>After</b> your children play with this awe...,Magnetic Before and After Activity Game - Supe...,Super Duper Publications,"[Sturdy, Colorful, Magnetic Game Board (14"" x ...",1586503839,Toys_and_Game
277,"[Toys & Games, Games, Card Games]",[],Duh: It Will Make You Feel Smart Game,n/a,[],0974889148,Toys_and_Game


Video_Game (982, 6)


,category,description,title,brand,feature,asin,domain
125,"[Video Games, PC]",[The Limited Edition game features six special...,Battlefield Bad Company 2 Limited Edition,Battlefield Bad Company 2 Limited Edition,[],8990289440,Video_Game
345,"[Video Games, Retro Gaming & Microconsoles, Ni...","[Nintendo 64, <i>Star Wars: Episode 1 Racer</i...",Star Wars - Episode I - Racer,Nintendo,[lose. not with box],B00000J2OO,Video_Game


In [57]:
df = pd.concat(dfs)

df.shape

(22991, 7)

In [59]:
df.sample(3)

,category,description,title,brand,feature,asin,domain
185,"[Industrial & Scientific, Tapes, Adhesives & S...",[America's #1 selling latex caulk. ALEXPLUS is...,DAP INC 18152 10.1oz White Alex Plus Acrylic L...,DAP,"[Sturdy and secure construction, Perfect for y...",B00002ND6L,Industrial_and_Scientific
868,"[Office Products, Office &amp; School Supplies...",[This Lang Heart &amp; Home 2016 Wall Calendar...,The Lang Heart &amp; Home 2016 Wall Calendar f...,Lang,"[Artwork from hand painted originals, Brass gr...",0741255162,Office_Product
491,"[Grocery & Gourmet Food, Olives, Pickles & Rel...",[This is an authentic German sauerkraut and is...,Kuehne former Gundelsheim Barrel Sauerkraut (1...,Gundelsheim,[],B0000D9MT2,Grocery_and_Gourmet_Food
